<a href="https://www.kaggle.com/code/averma111/pytorch-ps-s3e14-v1?scriptVersionId=128617028" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%%capture
!pip install flaml

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [26]:
ROOT_DIR='/kaggle/input/playground-series-s3e14'
ORIGINAL_DIR='/kaggle/input/wild-blueberry-yield-prediction/Data in Brief/Data in Brief/WildBlueberryPollinationSimulationData.csv'
train_file = 'train.csv'
origianl ='WildBlueberryPollinationSimulationData.csv'

In [27]:
def preprocessing():
    train = pd.read_csv(ROOT_PATH+'/'+train_file)
    original_df = pd.read_csv(ORIGINAL_DIR)
    train.drop('id',axis=1,inplace=True)
    original_df.drop('Row#',axis=1,inplace=True)
    df_full = pd.concat([train, original_df])
    for col in original_df.columns:
        df_full[col] = df_full[col].astype('float64')
    df_full = df_full.drop_duplicates()
    
    return df_full

In [28]:
def summary(text, df):
    print(f'{text} shape: {df.shape}')
    summ = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summ['null'] = df.isnull().sum()
    summ['unique'] = df.nunique()
    summ['min'] = df.min()
    summ['median'] = df.median()
    summ['max'] = df.max()
    summ['mean'] = df.mean()
    summ['std'] = df.std()
    summ['inf'] = np.isinf(df).sum().sum()
    summ['duplicate'] = df.duplicated().sum()
    return summ

In [29]:
df_full=preprocessing()

NameError: name 'ROOT_PATH' is not defined

In [30]:
summary('full',df_full)

NameError: name 'df_full' is not defined

In [75]:
def feature_engineering(df):
    df["fruit_seed"] = df["fruitset"] * df["seeds"]
    df['insects'] = df['honeybee'] + df['bumbles'] + df['andrena'] + df['osmia']
    return df

In [76]:
# Train Data
class TrainData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [77]:
def load_data(df_final):
    y = df_final['yield']
    df_final.drop(columns=['yield'],axis=1,inplace=True)
    X = df_final.copy()
    scaler = StandardScaler()
    numerical_cols = list(X.select_dtypes(include=['int','float']))
    X_numerical = scaler.fit_transform(X[numerical_cols].values)
    y=y.numpy()
    X_train,X_test,y_train,y_test = train_test_split(X_numerical,y,test_size=0.2,random_state=42)
    train_data = TrainData(torch.tensor(data=X_train,dtype=torch.float32,requires_grad=True),
                       torch.tensor(data=y_train,dtype=torch.float32,requires_grad=True))
    val_data = TrainData(torch.tensor(data=X_test,dtype=torch.float32,requires_grad=True),
                       torch.tensor(data=y_test,dtype=torch.float32,requires_grad=True))
    return train_data,val_data
    

In [78]:
class RegressionBlueBerryNNet(torch.nn.Module):
    def __init__(self,input_features):
        super(RegressionBlueBerryNNet,self).__init__()
        self.input_layer = torch.nn.Linear(input_features,1)
        self.relu = torch.nn.ReLU()
        
    def forward(self,inputs):
        x = self.relu(self.input_layer(inputs))
        return x

In [79]:
df_final=feature_engineering(df_full)

In [ ]:
from ray import tune
def train_nnet(config, checkpoint_dir=None, data_dir=None):

    if "l1" not in config:
        logger.warning(config)
    #net = Net(2**config["l1"], 2**config["l2"])
    net = RegressionBlueBerryNNet(16)

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.L1Loss()
    optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)

    if checkpoint_dir:
        checkpoint = os.path.join(checkpoint_dir, "checkpoint")
        model_state, optimizer_state = torch.load(checkpoint)
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    trainset,valset= load_data(df_final)

    trainloader = DataLoader(dataset=trainset, batch_size=int(2**config["batch_size"]), shuffle=True)
    valloader  =  DataLoader(dataset=valset, batch_size=int(2**config["batch_size"]))

    for epoch in range(int(round(config["num_epochs"]))):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = RegressionBlueBerryNNet(inputs.shape[1])
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = RegressionBlueBerryNNet(inputs.shape[1])
                #_, predicted = torch.max(outputs.data, 1)
                #total += labels.size(0)
                #correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1
                             
        with tune.checkpoint_dir(step=epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save(
                (net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps))
    print("Finished Training")

In [95]:
max_num_epoch = 100
config = {
    #"l1": tune.randint(2, 9),   # log transformed with base 2
   # "l2": tune.randint(2, 9),   # log transformed with base 2
    "lr": tune.loguniform(1e-4, 1e-1),
    "num_epochs": tune.loguniform(1, max_num_epoch),
    "batch_size": tune.randint(1, 5)    # log transformed with base 2
}

In [96]:
time_budget_s = 600     # time budget in seconds
gpus_per_trial = 0.5    # number of gpus for each trial; 0.5 means two training jobs can share one gpu
num_samples = 500       # maximal number of trials
np.random.seed(42)

2023-05-07 08:23:05,851	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::ImplicitFunc.train() (pid=4346, ip=172.19.2.2, repr=train_nnet)
  File "/opt/conda/lib/python3.10/site-packages/ray/tune/trainable/trainable.py", line 368, in train
    raise skipped from exception_cause(skipped)
  File "/opt/conda/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 337, in entrypoint
    return self._trainable_func(
  File "/opt/conda/lib/python3.10/site-packages/ray/tune/trainable/function_trainable.py", line 654, in _trainable_func
    output = fn()
  File "/opt/conda/lib/python3.10/site-packages/ray/tune/trainable/util.py", line 398, in inner
    return trainable(config, **fn_kwargs)
  File "/tmp/ipykernel_34/3449808912.py", line 6, in train_nnet
NameError: name 'Net' is not defined
2023-05-07 08:23:05,853	ERROR worker.py:399 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::ImplicitFunc.train() (pid=43

(autoscaler +1h17m37s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 0.5}. Add suitable node types to this cluster to resolve this issue.


In [97]:
import time
start_time = time.time()
result = flaml.tune.run(
    tune.with_parameters(train_nnet),
    config=config,
    metric="loss",
    mode="min",
    low_cost_partial_config={"num_epochs": 1},
    max_resource=max_num_epoch,
    min_resource=1,
    scheduler="asha",  # Use asha scheduler to perform early stopping based on intermediate results reported
    #resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
    local_dir='/kaggle/working/logs/',
    num_samples=num_samples,
    time_budget_s=time_budget_s,
    use_ray=True)

2023-05-07 08:23:27,138	WARNING optuna_search.py:332 -- You passed a `space` parameter to OptunaSearch that contained unresolved search space definitions. OptunaSearch should however be instantiated with fully configured search spaces only. To use Ray Tune's automatic search space conversion, pass the space definition as part of the `param_space` argument to `tune.Tuner()` instead.
[I 2023-05-07 08:23:27,141] A new study created in memory with name: optuna


(autoscaler +1h18m12s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 0.5}. Add suitable node types to this cluster to resolve this issue.
(autoscaler +1h18m48s) Error: No available node types can fulfill resource request {'CPU': 1.0, 'GPU': 0.5}. Add suitable node types to this cluster to resolve this issue.


2023-05-07 08:24:20,306	WARNING tune.py:146 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-05-07 08:24:22,324	ERROR tune.py:794 -- Trials did not complete: [train_nnet_23e6469f]
2023-05-07 08:24:22,325	INFO tune.py:798 -- Total run time: 55.18 seconds (55.12 seconds for the tuning loop).
2023-05-07 08:24:22,326	WARNING tune.py:804 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


In [82]:
print(f"#trials={len(result.trials)}")
print(f"time={time.time()-start_time}")
best_trial = result.get_best_trial("loss", "min", "all")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
    best_trial.metric_analysis["loss"]["min"]))
print("Best trial final validation accuracy: {}".format(
    best_trial.metric_analysis["accuracy"]["max"]))

best_trained_model = Net(2**best_trial.config["l1"],
                         2**best_trial.config["l2"])
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if gpus_per_trial > 1:
        best_trained_model = nn.DataParallel(best_trained_model)
best_trained_model.to(device)

checkpoint_value = getattr(best_trial.checkpoint, "dir_or_data", None) or best_trial.checkpoint.value
checkpoint_path = os.path.join(checkpoint_value, "checkpoint")

model_state, optimizer_state = torch.load(checkpoint_path)
best_trained_model.load_state_dict(model_state)

test_acc = _test_accuracy(best_trained_model, device)
print("Best trial test set accuracy: {}".format(test_acc))

NameError: name 'result' is not defined